## Preprocessing

In [21]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import load_model

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [3]:
# Determine the number of unique values in each column.
print(application_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T9", "T13", "T12", "T2", "T14", "T25", "T15", "T29", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C5200        1
C2170        1
C2380        1
C1728        1
C2190        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#I've decided to bin all values smaller than 50
classification_counts_smaller_than_50 = classification_counts[classification_counts < 50]

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_counts_smaller_than_50.index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      391
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df2 = pd.get_dummies(application_df)

#double check new columns
columns = application_df2.columns.values.tolist()
columns

['STATUS',
 'ASK_AMT',
 'IS_SUCCESSFUL',
 'APPLICATION_TYPE_Other',
 'APPLICATION_TYPE_T10',
 'APPLICATION_TYPE_T19',
 'APPLICATION_TYPE_T3',
 'APPLICATION_TYPE_T4',
 'APPLICATION_TYPE_T5',
 'APPLICATION_TYPE_T6',
 'APPLICATION_TYPE_T7',
 'APPLICATION_TYPE_T8',
 'AFFILIATION_CompanySponsored',
 'AFFILIATION_Family/Parent',
 'AFFILIATION_Independent',
 'AFFILIATION_National',
 'AFFILIATION_Other',
 'AFFILIATION_Regional',
 'CLASSIFICATION_C1000',
 'CLASSIFICATION_C1200',
 'CLASSIFICATION_C1270',
 'CLASSIFICATION_C1280',
 'CLASSIFICATION_C1300',
 'CLASSIFICATION_C1700',
 'CLASSIFICATION_C2000',
 'CLASSIFICATION_C2100',
 'CLASSIFICATION_C2700',
 'CLASSIFICATION_C2800',
 'CLASSIFICATION_C3000',
 'CLASSIFICATION_C4000',
 'CLASSIFICATION_C5000',
 'CLASSIFICATION_C7000',
 'CLASSIFICATION_C7100',
 'CLASSIFICATION_Other',
 'USE_CASE_CommunityServ',
 'USE_CASE_Heathcare',
 'USE_CASE_Other',
 'USE_CASE_Preservation',
 'USE_CASE_ProductDev',
 'ORGANIZATION_Association',
 'ORGANIZATION_Co-operative

In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df2['IS_SUCCESSFUL'].values
X = application_df2.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="elu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                4320      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,781
Trainable params: 6,781
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 774us/step - loss: 0.5270 - accuracy: 0.7441
Epoch 2/100
804/804 [==============================] - 1s 790us/step - loss: 0.5273 - accuracy: 0.7439
Epoch 3/100
804/804 [==============================] - 1s 868us/step - loss: 0.5271 - accuracy: 0.7441
Epoch 4/100
804/804 [==============================] - 1s 806us/step - loss: 0.5276 - accuracy: 0.7435
Epoch 5/100
804/804 [==============================] - 1s 798us/step - loss: 0.5273 - accuracy: 0.7436
Epoch 6/100
804/804 [==============================] - 1s 816us/step - loss: 0.5271 - accuracy: 0.7444
Epoch 7/100
804/804 [==============================] - 1s 805us/step - loss: 0.5273 - accuracy: 0.7430
Epoch 8/100
804/804 [==============================] - 1s 806us/step - loss: 0.5270 - accuracy: 0.7436
Epoch 9/100
804/804 [==============================] - 1s 818us/step - loss: 0.5268 - accuracy: 0.7445
Epoch 10/100
804/804 [==============================] - 1s 808us/step - l

804/804 [==============================] - 1s 791us/step - loss: 0.5269 - accuracy: 0.7444
Epoch 80/100
804/804 [==============================] - 1s 790us/step - loss: 0.5266 - accuracy: 0.7446
Epoch 81/100
804/804 [==============================] - 1s 791us/step - loss: 0.5266 - accuracy: 0.7448
Epoch 82/100
804/804 [==============================] - 1s 790us/step - loss: 0.5267 - accuracy: 0.7441
Epoch 83/100
804/804 [==============================] - 1s 795us/step - loss: 0.5268 - accuracy: 0.7440
Epoch 84/100
804/804 [==============================] - 1s 789us/step - loss: 0.5263 - accuracy: 0.7444
Epoch 85/100
804/804 [==============================] - 1s 790us/step - loss: 0.5268 - accuracy: 0.7446
Epoch 86/100
804/804 [==============================] - 1s 788us/step - loss: 0.5265 - accuracy: 0.7443
Epoch 87/100
804/804 [==============================] - 1s 788us/step - loss: 0.5266 - accuracy: 0.7450
Epoch 88/100
804/804 [==============================] - 1s 800us/step - loss:

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5638 - accuracy: 0.7266 - 158ms/epoch - 590us/step
Loss: 0.5638013482093811, Accuracy: 0.7266472578048706


In [26]:
# Export our model to HDF5 file
model = nn.save('AlphabetSoupCharity_Optimization.h5')
model = load_model('AlphabetSoupCharity_Optimization.h5')